All libraries used

In [ ]:
import threading
import cv2
from deepface import DeepFace
import os
import matplotlib as plt
import csv
from datetime import date

Screen setup

In [ ]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

path = 'assets'  # Change 'asset' to 'assets' for consistency with your code
images = os.listdir(path)
ref_img = [cv2.imread(os.path.join(path, i), cv2.IMREAD_COLOR) for i in images]
face_match = False
image_name = None

# Create a CSV file to store attendance
csv_filename = f'attendance_{date.today()}.csv'

Checks face value

In [ ]:
def check_face(frame):
    global face_match, image_name
    face_match = False
    try:
        for ref, image_name in zip(ref_img, images):
            result = DeepFace.verify(frame, ref.copy())
            if result['verified']:
                cv2.putText(frame, 'Success', (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
                face_match = True
                break
            else:
                cv2.putText(frame, 'No match!', (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
    except ValueError:
        pass


Attendance marking

In [ ]:
def create_attendance_csv():
    with open(csv_filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Name", "Date"])


def record_attendance(filename):
    # Extract the name from the filename
    name = os.path.splitext(filename)[0]

    with open(csv_filename, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow([name, date.today()])


# Check if the attendance CSV file exists, if not, create it
if not os.path.exists(csv_filename):
    create_attendance_csv()

FPS and threading

In [ ]:
fps = 0

def update_fps():
    global fps
    while True:
        fps = cap.get(cv2.CAP_PROP_FPS)


threading.Thread(target=update_fps).start()

# Parallelize the face detection and recognition
counter = 0

End 

In [ ]:
while True:
    ret, frame = cap.read()
    if ret:
        if counter % 3 == 0:
            threading.Thread(target=check_face, args=(frame.copy(),)).start()

        counter += 1

        if face_match:
            cv2.putText(frame, 'Success', (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
            record_attendance(image_name)
            face_match = False  # Reset the flag
        else:
            cv2.putText(frame, 'No match!', (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)

        cv2.imshow('video', frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cv2.destroyAllWindows()